In [4]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model = "gpt-5")

class Country(BaseModel):
    """Info about the country"""

    name: str = Field(description="Country name")
    language: str = Field(description="Language of the country")
    capital: str = Field(description="Capital of the country")

structured_llm = llm.with_structured_output(Country)
print(structured_llm)

first=RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000225AC388450>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000225A8A3F710>, root_client=<openai.OpenAI object at 0x00000225ABAD7B50>, root_async_client=<openai.AsyncOpenAI object at 0x00000225AC12E110>, model_name='gpt-5', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'response_format': <class '__main__.Country'>, 'structured_output_format': {'kwargs': {'method': 'json_schema'}, 'schema': {'type': 'function', 'function': {'name': 'Country', 'description': 'Info about the country', 'parameters': {'properties': {'name': {'description': 'Country name', 'type': 'string'}, 'language': {'description': 'Language of the country', 'type': 'string'}, 'capital': {'description': 'Capital of the country', 'type': 'string'}}, 'required': ['name', 'language', 'capital'], 'type': 'object'}}}}}, config={}, config_factories=[]) middle=[] last

In [5]:
structured_llm.invoke("Tell me about Sri Lanka")

Country(name='Sri Lanka', language='Sinhala and Tamil', capital='Sri Jayawardenepura Kotte')

When we don't need validation for the response from llm.

In [6]:
from typing_extensions import Annotated, TypedDict
from typing import Optional

class Joke(TypedDict):
    """Joke for the user"""
    setup: Annotated[str, ..., "The setup of the joke"]
    #Alternate methods for setup
    # setup: str
    # setup: Annotated[str, ...]
    # setup: Annotated[str, "foo"]

    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "Rate how funny is the joke from 1 to 10"]

structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke")

{'setup': 'I told my suitcase we aren’t going on vacation this year.',
 'punchline': 'Now I’m dealing with emotional baggage.',
 'rating': 7}

In [7]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)
structured_llm.invoke("Tell me a joke")

{'setup': "Why don't programmers like nature?",
 'punchline': 'It has too many bugs.',
 'rating': 7}